In [1]:
using Plots
using WebIO
using Interact
using Distributions
gr();

WebIO._IJuliaInit()

# Estimating π

$$\pi = 4\int_0^1 \int_0^1 \mathbf{1}_{\sqrt{x^2+y^2}\leq 1}(x, y) \, dx \, dy$$


$$\pi \approx \frac{4}{N} \sum_{i=1}^N \mathbf{1}_{\sqrt{x^2+y^2}\leq 1}(X_i, Y_i)$$

In [5]:
mc_sample(n=1000) = [(rand(), rand()) for _ in 1:n]

in_circle(xy) = sqrt(sum(xy.^2)) <= 1.0

estimate_pi(samples=mc_sample()) = sum(in_circle, samples)/length(samples)*4.0

estimate_pi (generic function with 2 methods)

In [6]:
function scatter_pi(samples=mc_sample();
                    title="π ≈ $(estimate_pi(samples)); n = $(length(samples))",
                    )
    scatter(samples,
        color=map(xy -> in_circle(xy) ? "red" : "blue", samples),
        title = title,
        xlim=(0,1),
        ylim=(0,1),
        aspect_ratio=:equal,
        legend=nothing
    )
end

scatter_pi (generic function with 2 methods)

In [7]:
gr()
@manipulate for n in exp10.(1:0.5:4)
    scatter_pi(mc_sample(n))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["n"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 7, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(4), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/zach/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/home/zach/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/home/zach/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x00007f164d7ac4c0, Task (runnable) @0x00007f164d7ac4c0), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 139733744730448, 139779710648320)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"10.0\",\"31.6228\",\"100.0\",\"316.228\",\"1000.0\",\"3162.28\",\"10000.0\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"40170080596

In [11]:
function mc_pi_traj(n=1000, m=100)
    catted_samples = [mc_sample(m)]
    for _ in 1:n-1
        push!(catted_samples, vcat(last(catted_samples), mc_sample(m)))
    end
    return m:m:m*n, estimate_pi.(catted_samples)
end

mc_pi_traj (generic function with 3 methods)

In [12]:
plotly()
plot([1,100_000], [pi, pi])
plot!(mc_pi_traj()... , xlabel="samples", legend=nothing)

In [13]:
plotly()
p = plot([1,100_000], [pi,pi])
for _ in 1:10
    plot!(p, mc_pi_traj()... , xlabel="samples", legend=nothing)
end
p

# Quantifying Error

In [ ]:
gr()
plots = [scatter_pi(mc_sample(100); title="") for _ in 1:16]
plot(plots..., layout=16)

In [ ]:
M = 1000
@manipulate for N in round.(Int, exp10.(0:0.5:5))
    histogram([estimate_pi(mc_sample(N)) for _ in 1:M],
        bins= N<1000 ? (0:0.1:4.1) : (3:0.02:3.3),
        label="Q_$(N)"
    )
end

# Central Limit Theorem

In [ ]:
M = 1000
@manipulate for N in exp10.(0:0.5:5)
    h = histogram([estimate_pi(mc_sample(N)) for _ in 1:M],
        bins=0:0.1:4.1,
        label="Estimator Sample Histogram"
    )
    plot!(h, x->M*0.1*pdf(Normal(pi, 4*sqrt(pi/4-pi^2/16)/sqrt(N)), x),
          label="True Estimator Distribution",
        legend=:left
    )
end

# SEM Approximation

In [ ]:
@manipulate for N in exp10.(1:0.5:5)
    samples = mc_sample(N)
    plot(x->pdf(Normal(estimate_pi(samples), 4*std(in_circle.(samples))/sqrt(N)), x),
         label="SEM Approximation"
    )
    plot!(x->pdf(Normal(pi, 4*sqrt(pi/4-pi^2/16)/sqrt(N)), x),
          label="True Estimator Distribution",
        legend=:left,
        xlim=(2.5, 3.8)
    )
end

In [ ]:
@manipulate for k in 1:10
    k
end